In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
def taxi_data(df):
    """This function return new table, which will be have 2500 regions to columns and plus region"""
    
    # delete values with:
    # trip duration  = 0
    indx = df[df[' pickup_datetime'] == df[' dropoff_datetime']].index
    #indx = df[df.tpep_pickup_datetime == df.tpep_dropoff_datetime].index   # for 2015-2016
    df.drop(indx, inplace=True)
    # 0 passager
    df.drop(df[df[' passenger_count'] == 0].index, inplace=True)   # 2014
    #df.drop(df[df['passenger_count'] == 0].index, inplace=True)
    # 0 distance
    #df.drop(df[df['trip_distance'] == 0].index, inplace=True)
    df.drop(df[df[' trip_distance'] == 0].index, inplace=True)       # 2014
    # Удалить координатами начала, не попадающими в прямоугольник Нью-Йорка от -74.25559 до -73.70001 градусов долготы 
    # и от 40.49612 до 40.91553 широты
    df = df[(df[' pickup_latitude'] <= 40.91553) & (df[' pickup_latitude'] >= 40.49612)]
    df = df[(df[' pickup_longitude'] >= -74.25559) & (df[' pickup_longitude'] <= -73.70001)]  # 2014
    #df = df[(df['pickup_latitude'] <= 40.91553) & (df['pickup_latitude'] >= 40.49612)]
    #df = df[(df['pickup_longitude'] >= -74.25559) & (df['pickup_longitude'] <= -73.70001)]
    
    # Change type object to datetime
    #df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df[' pickup_datetime'] = pd.to_datetime(df[' pickup_datetime'])    # 2014

    # Отбросьте минуты и секунды во времени начала поездки
    df['time'] = df[' pickup_datetime'].apply(lambda t: t.strftime('%Y-%m-%d %H'))    # For 2014
    #df['time'] = df.tpep_pickup_datetime.apply(lambda t: t.strftime('%Y-%m-%d %H'))
    # Add counter (1) for every trip 
    df['count_trip'] = 1
    
    # Start and finish interval of New York. And then cut it into 2500 pieces
    longitude = np.linspace(-74.25559,-73.70001, 51)
    latitude = np.linspace(40.49612,40.91553, 51)
    
    # 2014
    statistic = stats.binned_statistic_2d(df[' pickup_longitude'].values, df[' pickup_latitude'].values, 
                                      df['vendor_id'], statistic='count', bins=[longitude, latitude], expand_binnumbers=True) 
    #statistic = stats.binned_statistic_2d(df['pickup_longitude'].values, df['pickup_latitude'].values, 
                                      #df['VendorID'], statistic='count', bins=[longitude, latitude], expand_binnumbers=True) 

    df['region'] = (statistic[3][0] - 1) * 50 + statistic[3][1]
    
    # Create new table
    tab = df.pivot_table(values='count_trip', index=['time'], columns=['region'], aggfunc='count', fill_value=0)
    # Take out indexes 
    tab.reset_index(inplace=True)
    
    # Create zero-matrix for new table
    zeros = pd.DataFrame(np.zeros((tab.shape[0], 2500)), columns=np.arange(1, 2501))
    # Connect two tables and get new table which has size = 744*2501
    table = pd.merge(zeros, tab, how='outer').fillna(0)  #, left_index=True, right_index=True)
    drop_ind = table.index[:tab.shape[0]]
    table.drop(drop_ind, inplace=True)
    table.index = np.arange(tab.shape[0])
    table.iloc[:, :-1] = table.iloc[:, :-1].astype('int')
    
    return table

In [3]:
# df = pd.read_csv('yellow_tripdata_2016-05.csv')
# NYC = taxi_data(df)
# NYC

,1,2,3,4,5,6,7,8,9,10,...,2492,2493,2494,2495,2496,2497,2498,2499,2500,time
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-05-01 00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-05-01 01
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-05-01 02
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-05-01 03
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-05-01 04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-05-31 19
740,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-05-31 20
741,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-05-31 21
742,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-05-31 22


In [6]:
# NYC.to_csv('NYC_2016_05.csv')

In [8]:
jan_2014 = pd.read_csv('yellow_tripdata_2014-01.csv')

In [11]:
jan_2014.columns

Index(['vendor_id', ' pickup_datetime', ' dropoff_datetime',
       ' passenger_count', ' trip_distance', ' pickup_longitude',
       ' pickup_latitude', ' rate_code', ' store_and_fwd_flag',
       ' dropoff_longitude', ' dropoff_latitude', ' payment_type',
       ' fare_amount', ' surcharge', ' mta_tax', ' tip_amount',
       ' tolls_amount', ' total_amount'],
      dtype='object')

In [17]:
NY_2014_01 = taxi_data(jan_2014)

In [19]:
NY_2014_01.iloc[:, 1280:1285].sum(axis=0)

1281    574127
1282    769075
1283    172581
1284     70808
1285    211609
dtype: int64

In [20]:
NY_2014_01.to_csv('NYC_2014_01.csv')

In [57]:
march_2014 = pd.read_csv('yellow_tripdata_2014-03.csv')
march_2014

C:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2014-03-01 01:07:38,2014-03-01 01:16:26,1,2.0,-73.951504,40.714113,1,N,-73.944278,40.690677,CRD,9.0,0.5,0.5,2.0,0.0,12.0
1,CMT,2014-03-01 01:08:03,2014-03-01 01:12:51,2,1.2,-74.004735,40.742173,1,N,-73.992228,40.748210,CRD,6.0,0.5,0.5,1.0,0.0,8.0
2,CMT,2014-03-01 01:08:51,2014-03-01 01:13:18,3,0.5,-73.949302,40.707569,1,N,-73.951654,40.714250,CRD,5.0,0.5,0.5,1.2,0.0,7.2
3,CMT,2014-03-01 01:09:20,2014-03-01 01:24:18,3,3.5,-73.993539,40.721300,1,N,-73.961437,40.760412,CRD,14.0,0.5,0.5,3.0,0.0,18.0
4,CMT,2014-03-01 01:09:46,2014-03-01 01:22:34,1,1.8,-73.987881,40.744579,1,N,-74.004384,40.722775,CRD,10.5,0.5,0.5,1.0,0.0,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15428122,CMT,2014-03-18 18:14:19,2014-03-18 18:27:22,1,0.3,-73.987367,40.739568,1,N,-74.008448,40.743081,CRD,3.5,1.0,0.5,6.0,0.0,11.0
15428123,CMT,2014-03-18 10:12:33,2014-03-18 10:28:09,1,3.3,-73.973571,40.789391,1,N,-73.958701,40.760576,CRD,13.5,0.0,0.5,4.2,0.0,18.2
15428124,CMT,2014-03-18 09:02:37,2014-03-18 09:16:29,1,6.2,-73.977260,40.749512,1,N,-74.013530,40.714990,CRD,19.5,0.0,0.5,1.0,0.0,21.0
15428125,CMT,2014-03-18 10:10:19,2014-03-18 10:19:25,1,1.7,-73.981583,40.769048,1,N,-73.962727,40.773948,CRD,8.5,0.0,0.5,1.8,0.0,10.8


In [65]:
june_2014 = pd.read_csv('yellow_tripdata_2014-06.csv')

In [6]:
jule_2014 = pd.read_csv('yellow_tripdata_2014-07.csv')

In [7]:
NY_2014_07 = taxi_data(jule_2014)

In [8]:
NY_2014_07.to_csv('NYC_2014_07.csv')

In [14]:
may_2014 = pd.read_csv('yellow_tripdata_2014-05.csv')

In [15]:
NY_2014_05 = taxi_data(may_2014)
NY_2014_05.iloc[:, 1280:1285].sum(axis=0)

1281    595511
1282    766631
1283    199087
1284     79332
1285    223992
dtype: int64

In [16]:
NY_2014_05.to_csv('NYC_2014_05.csv')

In [3]:
august_2014 = pd.read_csv('yellow_tripdata_2014-08.csv')

C:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
NY_2014_08 = taxi_data(august_2014)

In [5]:
NY_2014_08.to_csv('NYC_2014_08.csv')

In [6]:
NY_2014_08.iloc[:, 1280:1285].sum(axis=0)

1281    538069
1282    700758
1283    155203
1284     64244
1285    160673
dtype: int64

In [7]:
september_2014 = pd.read_csv('yellow_tripdata_2014-09.csv')

In [8]:
NY_2014_09 = taxi_data(september_2014)
NY_2014_09.iloc[:, 1280:1285].sum(axis=0)

1281    528638
1282    707197
1283    176936
1284     67306
1285    196500
dtype: int64

In [ ]:
NY_2014_09.to_csv('NYC_2014_09.csv')

In [4]:
october_2014 = pd.read_csv('yellow_tripdata_2014-10.csv')

In [5]:
NY_2014_10 = taxi_data(october_2014)
NY_2014_10.iloc[:, 1280:1285].sum(axis=0)

1281    578140
1282    782742
1283    193841
1284     76073
1285    217873
dtype: int64

In [6]:
NY_2014_10.to_csv('NYC_2014_10.csv')

In [7]:
november_2014 = pd.read_csv('yellow_tripdata_2014-11.csv')

C:\Program Files\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
regions = pd.read_csv('regions.csv', sep=';')
regions.head(3)

In [25]:
import geojson
from geojson import Polygon, Feature, GeometryCollection, FeatureCollection, dump

In [38]:
def get_geojson(data):
    """This function returns"""
    featueres = []
    for i, row in data.iterrows():
        # west-north, east-north, east-south, west-south, west-north
        feature = Feature(geometry=Polygon([[(row.west, row.north), (row.east, row.north), (row.east, row.south), 
                                            (row.west, row.south), (row.west, row.north)]]), 
                                           id=int(row.region), properties={'region': int(row.region)})
        featueres.append(feature)
        
    return featueres

In [27]:
geo = get_geojson(regions)
# Save to file
with open('NYC_square.geojson', 'w') as f:
    dump(FeatureCollection(geo), f)

In [39]:
# Delete regions, where mean value trips < 5
regions['trip_mean'] = data.mean(axis=0).values 
taxi_mean_5 = regions[regions.trip_mean > 5]
# 
geo_5 = get_geojson(taxi_mean_5)
# Save to file
with open('NYC_sq_mean.geojson', 'w') as f:
    dump(FeatureCollection(geo_5), f)